

## Sparse Indentification of Non-Linear Dynamics (SINDY)

* Ref: J. Nathan Kutz book

It tries to leverage the fact that many dynamical systems 

$  \large \frac{d}{dt} x = f(x) $

have dynamics $ f $  with only a few active terms in the space of possible right-hand side functions

* for example: the Lorenz equations only have a few linear and quadratic interaction terms per equation


In [35]:

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D
from scipy import integrate



In [36]:

rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [12, 12]



In [37]:


## Simulate the Lorenz System

dt    = 0.01
T     = 50
t     = np.arange(dt,T+dt,dt)
beta  = 8/3
sigma = 10
rho   = 28
n     = 3




The goal is then to approximate $ f $ by a generalized LINEAR model

$   f(x) \approx  \sum_{k=1}^{p} \theta_k (x) \xi_k = \Theta (x)\xi   $

* with the fewest non-zero terms in $ \xi $ as possible.
* this is then solved via sparse regression
* time series data is collected and forms the data matrix





## First, time series data is collected and formed into matrix $ x $ 

* First, time series data is collected and formed into matrix $ x $ 



In [38]:

def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]



In [39]:

np.random.seed(123)
x0 = (-8,8,27)

x = integrate.odeint(
          lorenz_deriv, 
          x0, 
          t,
          rtol=10**(-12),
          atol=10**(-12)*np.ones_like(x0)
)


In [40]:

x.shape


(5000, 3)


## A similar matrix of derivatives is formed $ dx $

* A similar matrix of derivatives is formed $ dx $


In [41]:

## Compute Derivative
dx = np.zeros_like(x)

for j in range(len(t)):
    dx[j,:] = lorenz_deriv( x[j,:], 0, sigma, beta, rho)


In [42]:

dx.shape


(5000, 3)


## Library of candidate non-linear functions

* A library of non-linear functions 

$  \Theta(X)  $

may be constructed from the data in $ X $ such as:

$  \Theta (X) = [1 , X , X^2,  ... ,  X^d,  sin(X),   ... ]  $


* Here the matrix $ X^d $ denotes a matrix with column vectors given by all possible time series of dth-degree polynomials in the state $ x $. In general, this library of candidate functions is only limited by the imagination.  



In [43]:

## SINDy Function polynomials
## This generates the library of polynomials upto third order

def poolData(yin,nVars,polyorder):
    n = yin.shape[0]
    yout = np.zeros((n,1))
    
    # poly order 0
    yout[:,0] = np.ones(n)
    
    # poly order 1
    for i in range(nVars):
        yout = np.append(yout,yin[:,i].reshape((yin.shape[0],1)),axis=1)
    
    # poly order 2
    if polyorder >= 2:
        for i in range(nVars):
            for j in range(i,nVars):
                yout = np.append(yout,(yin[:,i]*yin[:,j]).reshape((yin.shape[0],1)),axis=1)
                
    # poly order 3
    if polyorder >= 3:
        for i in range(nVars):
            for j in range(i,nVars):
                for k in range(j,nVars):
                    yout = np.append(yout,(yin[:,i]*yin[:,j]*yin[:,k]).reshape((yin.shape[0],1)),axis=1)
    
    return yout



The dynamic system is now represented by 

$  X^{'} = \Theta (X) \Xi  $

* where each column $   \xi_k $ in $ \Xi  $ is a vector of coefficients determining the active terms in the kth row in $ X^{'} $

* The sequential threshold least-squares (STLS) algorithm is used to solve this 



## Optimization

* uses:  np.linalg.lstsq()

The loss has the form of:
    
$   \large    \xi_k = \underset{\xi^{'}_k }{argmin} \lVert X^{'}_k  -  \Theta (X) \xi^{'}_k  \rVert_2  + \lambda \lVert \xi^{'}_k \rVert $


In [44]:



def sparsifyDynamics(Theta, dXdt, lamb, n):
    
    Xi = np.linalg.lstsq(Theta, dXdt,rcond=None)[0] # Initial guess: Least-squares
    
    for k in range(10):
        smallinds = np.abs(Xi) < lamb # Find small coefficients
        Xi[smallinds] = 0                          # and threshold
        for ind in range(n):                       # n is state dimension
            biginds = smallinds[:,ind] == 0
            # Regress dynamics onto remaining terms to find sparse Xi
            Xi[biginds,ind] = np.linalg.lstsq(Theta[:,biginds],dXdt[:,ind],rcond=None)[0]
            
    return Xi


In [45]:

Theta = poolData(x, n, 3) # Up to third order polynomials

lamb  = 0.025 # sparsification knob lambda

Xi    = sparsifyDynamics(Theta, dx, lamb, n)

Xi.shape


(20, 3)


## The resulting data in Xi can be used to discover the Lorenz Equations

* Each row is a polinomial and the values below indicate which ones are connected to find the equations


In [46]:

print(Xi)


[[  0.           0.           0.        ]
 [-10.          28.           0.        ]
 [ 10.          -1.           0.        ]
 [  0.           0.          -2.66666667]
 [  0.           0.           0.        ]
 [  0.           0.           1.        ]
 [  0.          -1.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]]


In [47]:

Theta.shape


(5000, 20)

In [48]:

Theta


array([[ 1.00000000e+00, -8.00000000e+00,  8.00000000e+00, ...,
         1.72800000e+03,  5.83200000e+03,  1.96830000e+04],
       [ 1.00000000e+00, -6.48640305e+00,  7.80317052e+00, ...,
         1.56641344e+03,  5.16415787e+03,  1.70252155e+04],
       [ 1.00000000e+00, -5.13801768e+00,  7.56258900e+00, ...,
         1.40744562e+03,  4.57985524e+03,  1.49029375e+04],
       ...,
       [ 1.00000000e+00,  1.74442140e+00,  4.88108249e+00, ...,
         5.85115392e+02,  2.94397830e+03,  1.48124769e+04],
       [ 1.00000000e+00,  2.04379883e+00,  4.90288321e+00, ...,
         5.77019771e+02,  2.82505394e+03,  1.38312935e+04],
       [ 1.00000000e+00,  2.31782009e+00,  4.94677848e+00, ...,
         5.74536883e+02,  2.72689514e+03,  1.29425235e+04]])